# Tutorial about local density analysis

A local density is computed from the number of neighboring localizations
within a specified radius.

In [ ]:
from pathlib import Path

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Synthetic data

We simulate localization data that is homogeneously Poisson distributed (also described as complete spatial randomness, csr).

In [ ]:
rng = np.random.default_rng(seed=1)

In [ ]:
locdata_csr = lc.simulate_Poisson(intensity=1e-1, region=((0,100), (0,100)), seed=rng)

print('Data head:')
print(locdata_csr.data.head(), '\n')
print('Summary:')
locdata_csr.print_summary()
print('Properties:')
print(locdata_csr.properties)

We also simulate data that follows a Neyman-Scott distribution (blobs): 

In [ ]:
locdata_blob = lc.simulate_Thomas(parent_intensity=1e-3, region=((0, 100), (0, 100)), cluster_mu=100, cluster_std=5, seed=rng)

print('Data head:')
print(locdata_blob.data.head(), '\n')
print('Summary:')
locdata_blob.print_summary()
print('Properties:')
print(locdata_blob.properties)

### Scatter plot

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
locdata_csr.data.plot.scatter(x='position_x', y='position_y', ax=ax[0], color='Blue', s=1, alpha=0.1, label='locdata_csr')
locdata_blob.data.plot.scatter(x='position_x', y='position_y', ax=ax[1], color='Blue', s=1, alpha=0.1, label='locdata_blobs')
plt.tight_layout()
plt.show()

## Local densities

We determine the local density for each localization and plot the probability density function for local densities.

In [ ]:
lc.LocalDensity?

In [ ]:
ld_csr = lc.LocalDensity(radii=[3, 10, 30])
ld_csr.compute(locdata_csr)
ld_csr.results.describe()

In [ ]:
ld_blob = lc.LocalDensity(radii=[3, 10, 30])
ld_blob.compute(locdata_blob)
ld_blob.results.describe()

In [ ]:
radius = 3
fig, ax = plt.subplots(nrows=1, ncols=2)
locdata_csr.data.plot.scatter(x='position_x', y='position_y', ax=ax[0], color=ld_csr.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_csr')
locdata_blob.data.plot.scatter(x='position_x', y='position_y', ax=ax[1], color=ld_blob.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_blobs')
plt.tight_layout()
plt.show()

In [ ]:
radius = 10
fig, ax = plt.subplots(nrows=1, ncols=2)
locdata_csr.data.plot.scatter(x='position_x', y='position_y', ax=ax[0], color=ld_csr.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_csr')
locdata_blob.data.plot.scatter(x='position_x', y='position_y', ax=ax[1], color=ld_blob.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_blobs')
plt.tight_layout()
plt.show()

In [ ]:
radius = 30
fig, ax = plt.subplots(nrows=1, ncols=2)
locdata_csr.data.plot.scatter(x='position_x', y='position_y', ax=ax[0], color=ld_csr.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_csr')
locdata_blob.data.plot.scatter(x='position_x', y='position_y', ax=ax[1], color=ld_blob.results[radius], s=1, colormap='viridis', alpha=1, label='locdata_blobs')
plt.tight_layout()
plt.show()

In [ ]:
ld_csr.hist(alpha=0.5, density=True, bins=20);

In [ ]:
ld_blob.hist(alpha=0.5, density=True, bins=20);

In [ ]:
bins = np.arange(0, 1, 0.05)
ld_csr.hist(alpha=0.5, bins=bins)
ld_blob.hist(alpha=0.5, bins=bins);

## Local densities with boundary correction

We determine the local density for each localization with a boundary correction applied. For the correction local density values are normalized by the relative overlapp of the encircling region and the support region. 

In [ ]:
locdata = lc.simulate_uniform(n_samples=5_000, region=lc.Ellipse((0, 0), 100, 100))
locdata.region

In [ ]:
radius = 20

In [ ]:
ld = lc.LocalDensity(radii=[radius]).compute(locdata)

In [ ]:
ld.hist(bins=20);

In [ ]:
ld.results.index = locdata.data.index
df = locdata.dataframe.assign(local_density=ld.results[radius])
locdata = locdata.update(dataframe=df)
locdata.data.local_density.describe()

In [ ]:
fig, ax = plt.subplots()
lc.render_2d_mpl(locdata, ax=ax, loc_properties=["position_x", "position_y"], bin_size=1, other_property='local_density', rescale=lc.Trafo.NONE, vmin=0.2, vmax=0.7);
if locdata.region:
    locdata.region.plot(ax=ax, fill=False, color='Black');

In [ ]:
ld_2 = lc.LocalDensity(radii=[radius], boundary_correction=True).compute(locdata)

In [ ]:
ld_2.hist(bins=20);

In [ ]:
ld_2.results.index = locdata.data.index
df = locdata.dataframe.assign(local_density=ld_2.results[radius])
locdata = locdata.update(dataframe=df)
locdata.data.local_density.describe()

In [ ]:
fig, ax = plt.subplots()
lc.render_2d_mpl(locdata, ax=ax, loc_properties=["position_x", "position_y"], bin_size=1, other_property='local_density', rescale=lc.Trafo.NONE, vmin=0.2, vmax=0.7);
if locdata.region:
    locdata.region.plot(ax=ax, fill=False, color='White');